<a href="https://colab.research.google.com/github/rafaellccouto/postech_analise_vinhos_comercio/blob/main/Analise_vinhos_comercio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
#Analise dados Vinhos
#Bibliotecas de manipulação de dados
import pandas as pd
import numpy as np
#Bibliotecas de visualização de dados
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
#Base de dados
df_comercialização = pd.read_csv('Comercializacao.csv', sep=';', on_bad_lines='skip')
df_exportacao = pd.read_csv('Exportacao.csv', sep=';', on_bad_lines='skip')
df_importacao = pd.read_csv('Importacao.csv', sep=';', on_bad_lines='skip')
df_processamento = pd.read_csv('Processamento.csv', sep=';', on_bad_lines='skip')
df_producao = pd.read_csv('Producao.csv', sep=';', on_bad_lines='skip')

In [41]:
df_comercialização.head()

,id,control,Produto,1970,1971,1972,1973,1974,1975,1976,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,1,VINHO DE MESA,VINHO DE MESA,98327606,114399031,118377367,116617910,94173324,108031792,139238614,...,206404427,209198468,166769622,176059959,177186273,180446489,215557931,210012238,187939996,187016848
1,2,vm_Tinto,Tinto,83300735,98522869,101167932,98196747,77167303,91528090,116407222,...,178250072,182028785,146646365,154309442,155115499,158519218,189573423,185653678,165067340,165097539
2,3,vm_Rosado,Rosado,107681,542274,7770851,8425617,8891367,7261777,11748047,...,1419855,1409002,1391942,1097426,1972944,1265435,1394901,1931606,2213723,2520748
3,4,vm_Branco,Branco,14919190,15333888,9438584,9995546,8114654,9241925,11083345,...,26734500,25760681,18731315,20653091,20097830,20661836,24589607,22426954,20658933,19398561
4,5,VINHO FINO DE MESA,VINHO FINO DE MESA,4430629,4840369,5602091,7202830,7571802,8848303,14095648,...,20424983,20141631,19630158,15874354,14826143,15684588,24310834,27080445,21533487,18589310


In [42]:
df_exportacao.head()

,Id,País,1970,1970.1,1971,1971.1,1972,1972.1,1973,1973.1,...,2019,2019.1,2020,2020.1,2021,2021.1,2022,2022.1,2023,2023.1
0,1,Afeganistão,0,0,0,0,0,0,0,0,...,0,0,0,0,11,46,0,0,0,0
1,2,África do Sul,0,0,0,0,0,0,0,0,...,26,95,4,21,0,0,0,0,117,698
2,3,"Alemanha, República Democrática",0,0,0,0,4168,2630,12000,8250,...,3660,25467,6261,32605,2698,6741,7630,45367,4806,31853
3,4,Angola,0,0,0,0,0,0,0,0,...,345,1065,0,0,0,0,4068,4761,0,0
4,5,Anguilla,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
df_importacao.head()

,Id,País,1970,1970.1,1971,1971.1,1972,1972.1,1973,1973.1,...,2019,2019.1,2020,2020.1,2021,2021.1,2022,2022.1,2023,2023.1
0,1,Africa do Sul,0,0.0,0,0,0,0,0,0,...,1092042,3604038,627150,1701072,859169,2508140,738116,2266827,522733,1732850
1,2,Alemanha,52297,30498.0,34606,26027,134438,92103,111523,98638,...,101055,412794,136992,504168,106541,546967,92600,438595,102456,557947
2,3,Argélia,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Arábia Saudita,0,0.0,0,0,0,0,0,0,...,0,0,0,0,2510,8761,0,0,8,161
4,5,Argentina,19525,12260.0,24942,15022,104906,58137,116887,76121,...,16548931,54527380,22610267,66322932,26869241,79527959,27980574,87519642,25276991,83918138


In [44]:
df_processamento.head()

,id,control,cultivar,1970,1971,1972,1973,1974,1975,1976,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,1,TINTAS,TINTAS,10448228,11012833,10798824,8213674,17457849,22593885,20265190,...,29810706,29935627,13370866,32850915,26868514,nd,28003505,93296587,*,"35881118,23"
1,2,ti_Alicante Bouschet,Alicante Bouschet,0,0,0,0,0,0,0,...,1456305,1519576,908841,2040198,2103844,nd,2272985,811140,*,"4108858,21"
2,3,ti_Ancelota,Ancelota,0,0,0,0,0,0,0,...,937844,773526,179028,733907,492106,nd,481402,6513974,*,"783688,39"
3,4,ti_Aramon,Aramon,0,0,0,0,0,0,0,...,0,0,0,0,0,nd,0,0,*,0
4,5,ti_Alfrocheiro,Alfrocheiro,0,0,0,0,0,0,0,...,0,0,0,0,0,nd,0,0,*,0


In [45]:
df_producao.head()

,id,control,produto,1970,1971,1972,1973,1974,1975,1976,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,1,VINHO DE MESA,VINHO DE MESA,217208604,154264651,146953297,116710345,193875345,177401209,144565438,...,196173123,210308560,86319015,255015187,218375636,144629737,124200414,173899995,195031611,169762429
1,2,vm_Tinto,Tinto,174224052,121133369,118180926,88589019,146544484,144274134,118360170,...,157776363,169811472,75279191,1365957,188270142,121045115,103916391,146075996,162844214,139320884
2,3,vm_Branco,Branco,748400,1160500,1812367,243900,4138768,1441507,1871473,...,37438069,39557250,10727099,217527985,29229970,22032828,19568734,26432799,30198430,27910299
3,4,vm_Rosado,Rosado,42236152,31970782,26960004,27877426,43192093,31685568,24333795,...,958691,939838,312725,36121245,875524,1551794,715289,1391200,1988968,2531246
4,5,VINHO FINO DE MESA (VINIFERA),VINHO FINO DE MESA (VINIFERA),23899346,23586062,21078771,12368410,31644124,39424590,34500590,...,38464314,37148982,18070626,44537870,38707220,37615422,32516686,43474998,47511796,46268556


## Eliminação dos anos que não serão utilizados e unificação de semestres se houver

In [49]:
non_numeric_cols = ['id', 'control', 'Produto']
year_cols = [str(year) for year in range(2009, 2024)]

cols_to_select = non_numeric_cols + [col for col in year_cols if col in df_comercialização.columns]
comercializacao_filtrada = df_comercialização[cols_to_select].copy()

for col in year_cols:
    if col in comercializacao_filtrada.columns:
        comercializacao_filtrada[col] = pd.to_numeric(comercializacao_filtrada[col], errors='coerce').fillna(0)

print("Head of comercializacao_filtrada (filtered from 2009, nulls as 0):")
display(comercializacao_filtrada.head())

file_name = 'comercializacao_filtrada.csv'
comercializacao_filtrada.to_csv(file_name, index=False)

print(f"\nDataFrame saved to '{file_name}' and is ready for download.")


Head of comercializacao_filtrada (filtered from 2009, nulls as 0):


,id,control,Produto,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,1,VINHO DE MESA,VINHO DE MESA,234525979,221242945,230310468,206969571,221590810,206404427,209198468,166769622,176059959,177186273,180446489,215557931,210012238,187939996,187016848
1,2,vm_Tinto,Tinto,193004182,188649074,196562722,173964776,188033494,178250072,182028785,146646365,154309442,155115499,158519218,189573423,185653678,165067340,165097539
2,3,vm_Rosado,Rosado,2307580,2036928,1668823,1738134,1777648,1419855,1409002,1391942,1097426,1972944,1265435,1394901,1931606,2213723,2520748
3,4,vm_Branco,Branco,39214217,30556943,32078923,31266661,31779668,26734500,25760681,18731315,20653091,20097830,20661836,24589607,22426954,20658933,19398561
4,5,VINHO FINO DE MESA,VINHO FINO DE MESA,33080270,21390159,19967310,22469950,27912934,20424983,20141631,19630158,15874354,14826143,15684588,24310834,27080445,21533487,18589310



DataFrame saved to 'comercializacao_filtrada.csv' and is ready for download.


In [50]:
# Carrega o DataFrame original de exportação novamente para garantir um estado limpo
df_exportacao = pd.read_csv('Exportacao.csv', sep=';', on_bad_lines='skip')

# Define as colunas não-numéricas
non_numeric_cols_exportacao = ['Id', 'País']

# Remove as colunas de ano de 1970 a 2008 (incluindo as com sufixo '.1')
exportacao_cols_to_remove = []
for year in range(1970, 2009):
    year_str = str(year)
    year_str_dot_1 = year_str + '.1'
    if year_str in df_exportacao.columns:
        exportacao_cols_to_remove.append(year_str)
    if year_str_dot_1 in df_exportacao.columns:
        exportacao_cols_to_remove.append(year_str_dot_1)

df_exportacao = df_exportacao.drop(columns=exportacao_cols_to_remove, errors='ignore')

# Consolidar colunas de ano de 2009 a 2023
# Se houver 'YYYY.1', somar com 'YYYY' e manter apenas 'YYYY'. Se não, apenas garantir que 'YYYY' é numérico.
for year in range(2009, 2024):
    col_year = str(year)
    col_year_dot_1 = col_year + '.1'

    if col_year_dot_1 in df_exportacao.columns:
        # Converter ambas as colunas para numérico, tratando erros como NaN e preenchendo com 0
        df_exportacao[col_year] = (
            pd.to_numeric(df_exportacao[col_year], errors='coerce').fillna(0) +
            pd.to_numeric(df_exportacao[col_year_dot_1], errors='coerce').fillna(0)
        )
        # Remover a coluna com sufixo '.1' após a consolidação
        df_exportacao = df_exportacao.drop(columns=[col_year_dot_1])
    elif col_year in df_exportacao.columns:
        # Garantir que a coluna de ano existente seja numérica, mesmo que não haja '.1'
        df_exportacao[col_year] = pd.to_numeric(df_exportacao[col_year], errors='coerce').fillna(0)

# Criar o DataFrame final exportacao_filtrada
exportacao_filtrada = df_exportacao.copy()

# Exibir as primeiras linhas do DataFrame resultante
print("Head of exportacao_filtrada (filtrado a partir de 2009 e consolidado):")
display(exportacao_filtrada.head())

# Salvar o DataFrame em um arquivo CSV para download
file_name = 'exportacao_filtrada.csv'
exportacao_filtrada.to_csv(file_name, index=False)

print(f"\nDataFrame salvo em '{file_name}' e pronto para download.")


Head of exportacao_filtrada (filtrado a partir de 2009 e consolidado):


,Id,País,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,1,Afeganistão,0,0,0,0,0,0,0,0,0,0,0,0,57,0,0
1,2,África do Sul,0,0,0,0,0,0,0,0,0,0,121,25,0,0,815
2,3,"Alemanha, República Democrática",618568,166381,180220,64531,327677,975001,55460,82121,103169,56176,29127,38866,9439,52997,36659
3,4,Angola,139021,223448,82890,11694,10873,35306,18997,42749,71850,1186,1410,0,0,8829,0
4,5,Anguilla,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0



DataFrame salvo em 'exportacao_filtrada.csv' e pronto para download.


In [51]:
# Carrega o DataFrame original de importação para garantir um estado limpo
df_importacao = pd.read_csv('Importacao.csv', sep=';', on_bad_lines='skip')

# Define as colunas não-numéricas para df_importacao
non_numeric_cols_importacao = ['Id', 'País']

# Remove as colunas de ano de 1970 a 2008 (incluindo as com sufixo '.1')
importacao_cols_to_remove = []
for year in range(1970, 2009):
    year_str = str(year)
    year_str_dot_1 = year_str + '.1'
    if year_str in df_importacao.columns:
        importacao_cols_to_remove.append(year_str)
    if year_str_dot_1 in df_importacao.columns:
        importacao_cols_to_remove.append(year_str_dot_1)

df_importacao = df_importacao.drop(columns=importacao_cols_to_remove, errors='ignore')

# Consolidar colunas de ano de 2009 a 2023
# Se houver 'YYYY.1', somar com 'YYYY' e manter apenas 'YYYY'. Se não, apenas garantir que 'YYYY' é numérico.
for year in range(2009, 2024):
    col_year = str(year)
    col_year_dot_1 = col_year + '.1'

    if col_year_dot_1 in df_importacao.columns:
        # Converter ambas as colunas para numérico, tratando erros como NaN e preenchendo com 0
        df_importacao[col_year] = (
            pd.to_numeric(df_importacao[col_year], errors='coerce').fillna(0) +
            pd.to_numeric(df_importacao[col_year_dot_1], errors='coerce').fillna(0)
        )
        # Remover a coluna com sufixo '.1' após a consolidação
        df_importacao = df_importacao.drop(columns=[col_year_dot_1])
    elif col_year in df_importacao.columns:
        # Garantir que a coluna de ano existente seja numérica, mesmo que não haja '.1'
        df_importacao[col_year] = pd.to_numeric(df_importacao[col_year], errors='coerce').fillna(0)

# Criar o DataFrame final importacao_filtrada
importacao_filtrada = df_importacao.copy()

# Exibir as primeiras linhas do DataFrame resultante
print("Head of importacao_filtrada (filtrado a partir de 2009 e consolidado):")
display(importacao_filtrada.head())

# Salvar o DataFrame em um arquivo CSV para download
file_name = 'importacao_filtrada.csv'
importacao_filtrada.to_csv(file_name, index=False)

print(f"\nDataFrame salvo em '{file_name}' e pronto para download.")

Head of importacao_filtrada (filtrado a partir de 2009 e consolidado):


,Id,País,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,1,Africa do Sul,2424542,4566934,2643903,3763776,2460515,3756647,3739878,2265055,4380030,4701424,4696080,2328222,3367309,3004943,2255583
1,2,Alemanha,347951,977335,1131570,704410,528936,599120,453446,764719,399248,659946,513849,641160,653508,531195,660403
2,3,Argélia,0,0,0,0,0,0,0,43,0,0,0,0,0,0,0
3,4,Arábia Saudita,0,0,0,0,0,0,0,43,0,3812,0,0,11271,0,169
4,5,Argentina,52424481,69091847,75480227,70247401,62257509,67253277,60508242,59312871,67232582,68038960,71076311,88933199,106397200,115500216,109195129



DataFrame salvo em 'importacao_filtrada.csv' e pronto para download.
